# Exercise Sheet 9: Domain Adaptation on TweetQA using T5

In this exercise, you will evaluate T5 on question answering in the twitter domain, and then finetune the model.

You should complete the parts of the exercise that are marked as **TODO**.
A correctly completed **TODO** gives 2 bonus points. Partially correct answers give 1 bonus point.
Some **TODO**s are inside a comment in a code block: Here, you should complete the line of code.
Other **TODO**s are inside a text block: Here, you should write a few sentences to answer the question.

**Important:** Some students were under the impression that you have to complete a TODO in a _single_ line of code. That is not the case, you can use as many lines as you need.

**Submission deadline:** 19.01.2022

**Instructions for submission:** After completing the exercise, save a copy of the notebook as exercise9_tweetqa_MATRIKELNUMMER.ipynb, where MATRIKELNUMMER is your student ID number. Then upload the notebook to moodle (submission exercise sheet 9).

In order to understand the code, it can be helpful to experiment a bit during development, e.g., to print tensors or their shapes. But please remove these changes before submitting the notebook. If we cannot run your notebook, or if a print statement is congesting stdout too much, then we cannot grade it. 

To make the most of this exercise, you should try to read and understand the entire code, not just the parts that contain a **TODO**. If you have questions, write them down for the exercise, which will happen in the week after the submission deadline.

**CUDA:** You can use a GPU for this exercise (on colab: Runtime -> Change Runtime Type -> GPU). This is not mandatory, but it will speed up training epochs, thereby allowing you to test more hyperparameters.

# Libraries and Hyperparameters

In [2]:
!pip install -q transformers==4.15.0
!pip install -q datasets==1.2.0 #this is an version of datasets. The newest version has a bug for the specific dataset that we will use. HF has already written a fix, but have not merged it yet.

ERROR: keybert 0.2.0 has requirement numpy>=1.18.5, but you'll have numpy 1.18.0 which is incompatible.
ERROR: huggingface-hub 0.4.0 has requirement packaging>=20.9, but you'll have packaging 20.3 which is incompatible.


In [3]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, load_metric
import numpy as np
import torch

2022-01-16 18:35:46.509577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-16 18:35:46.509615: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
NUM_EPOCHS = 3 if torch.cuda.is_available() else 1
PERCENTILES = (95, 100) if torch.cuda.is_available() else (80, 100)

TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 64
WARMUP_STEPS = 200
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 100
LEARNING_RATE = 5e-05

In [5]:
torch.manual_seed(0)
print(torch.cuda.is_available())
if torch.cuda.is_available():
  torch.cuda.manual_seed(0)

False


# Data preprocessing

In this exercise, we will load the [tweet_qa dataset](https://huggingface.co/datasets/tweet_qa), which is a closed question answering dataset. Each data sample consists of a question, a tweet as context which contains the necessary information, and the correct answer. Luckily for us, T5 has already been trained on SQUAD, which is a similar task, but with the context consisting of snippets from Wikipedia articles. This means that we just need to bring our dataset into the right format, and then we can already take a look at how well T5 performs on it. After that, we will finetune on tweet_qa to improve performance (domain adaptation).

In [6]:
tokenizer = T5TokenizerFast.from_pretrained('t5-small')

Here, we are going to bring our dataset into the right format, tokenize it, and truncate the inputs using the same method as in the last exercise. Hint: to find out what format T5 expects questions and answers in, take a look at the [T5 paper](https://arxiv.org/pdf/1910.10683.pdf), specifically Appendix D.15. 

In [22]:
def reformat_for_t5(example):
  # TODO: bring the question and context into the format that T5 is used to from SQUAD, for each example
  index = 0
  return_data = ''
  return_data += 'question: '
  return_data += example['Question']
  return_data += '  '
  return_data += 'context: '
  for answer in example['Tweet']:
    return_data += answer
  example['src_texts'] = return_data
  return example

def get_max_length(tokenizer, train_dataset, column, percentile):
  def get_lengths(batch):
    return tokenizer(batch, padding=False, return_length=True)

  lengths = train_dataset.map(get_lengths, input_columns=column, batched=True)['length']
  return int(np.percentile(lengths, percentile)) +1

ANSWER_SEP = ':::::'
dataset = load_dataset('tweet_qa')
dataset = dataset.map(reformat_for_t5)
dataset = dataset.map(lambda x: {'tgt_texts': ANSWER_SEP.join(x['Answer'])}) 
#note: this is a hack: The Seq2SeqTrainer we will use later does not allow us to pass multiple labels to it. However, our dataset contains multiple correct answers for each question in the validation part. Therefore we concatenate them here so we can later split them in compute_metrics. Don't worry about this too much.

max_length = get_max_length(tokenizer, dataset['train'], 'src_texts', PERCENTILES[0])
max_target_length = get_max_length(tokenizer, dataset['train'], 'tgt_texts', PERCENTILES[1])

def tokenize(batch):
  # TODO: call the tokenizer once for the inputs and once for the targets. Make sure to use the correct max_length, use the correct padding, and enable truncation.
  inputs =  tokenizer(batch['src_texts'], max_length=max_length, padding=True, truncation=True)
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(batch['tgt_texts'], padding=True, max_length=max_target_length, truncation=True)
  inputs["labels"] = targets["input_ids"]
  return inputs
  
dataset = dataset.map(tokenize, batched=True)
dataset.set_format('torch', columns=['input_ids', 'labels', 'attention_mask'])

Using custom data configuration default
Reusing dataset tweet_qa (/home/min20120907/.cache/huggingface/datasets/tweet_qa/default/1.0.0/1d584f8fe9cb9d3d8d9ab63a36a25716b482850adc0e94f5d63c081ea2f78f01)


# Defining Metrics

We are going to evaluate with the same metrics used by SQUAD, Exact Match and Bag-of-Words F1. They're already available in 🤗 datasets, but we need to get our predictions and answers into shape first.

In [ ]:
def squad_metrics(eval_prediction):
  # TODO: use the batch_decode function of the tokenizer to decode the predictions and the label_ids (hint: they're both in the only argument of this function). Remember to set skip_special_tokens so that we don't generate padding tokens.
  predictions = None
  answers = None
  
  answers = [answer.split(ANSWER_SEP) for answer in answers]
  assert all([len(ans) == 2 for ans in answers])
  
  predictions = [{'id': str(i), 'prediction_text': pred.strip().lower()} \
                 for i, pred in enumerate(predictions)]
  references = [{'id': str(i), 'answers': {'text': ans, 'answer_start': []}} \
                for i, ans in enumerate(answers)]

  #TODO: load the metric "squad" using the function we've imported at the top of the notebook. 
  #TODO: use the add_batch method to pass our predictions and references to the metric
  #TODO: compute the metric
  return None

# Training

In [23]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [26]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_dir='./logs/',
    evaluation_strategy="steps",
    logging_steps=LOGGING_STEPS,
    learning_rate=LEARNING_RATE,
    predict_with_generate=True,
)

#TODO: instantiate the Seq2SeqTrainer. Remember to pass all the objects we've constructed so far that we can use here.

trainer = Seq2SeqTrainer(training_args)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


AttributeError: type object 'Seq2SeqTrainer' has no attribute 'TrainingArguments'

# Evaluation before Domain Adaptation

First, let's test how T5 performs on tweet_qa without domain adaptation. It should work fairly well, given that it's been trained on SQUAD, which is a very similar task, just on a different domain. For evaluation, we'll perform beam search during generation with a beam size of 2.

In [ ]:
print(trainer.evaluate(num_beams=2))

# Domain Adaptation

Now let's see how much we can improve our performance by finetuning on our in-domain data:

In [ ]:
trainer.train()

In [ ]:
print(trainer.evaluate())
print(trainer.evaluate(num_beams=2))

# Report

**TODO:** Write a brief report of your results. Include the hyperparameters that you used, the results T5 achieved on the dataset before and after domain adaptation, and why the evaluation during training may have returned different results than the separate evaluations before and after training. Also report on the different evaluation speeds resulting from this, and give a brief explanation why finetuning on in-domain data was helpful for performance.